<a href="https://colab.research.google.com/github/mohammedradman1/DSE_Term_Project/blob/main/DSE_Term_Project_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Obtaining Data from Git Repository

In [2]:
# obtaining the data from Git Repository
!wget https://github.com/mohammedradman1/DSE_Term_Project/blob/main/VIW_FNT.csv


--2023-12-18 19:44:01--  https://github.com/mohammedradman1/DSE_Term_Project/blob/main/VIW_FNT.csv
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4371 (4.3K) [text/plain]
Saving to: ‘VIW_FNT.csv’

VIW_FNT.csv         100%[===================>]   4.27K  --.-KB/s    in 0s      

2023-12-18 19:44:01 (51.6 MB/s) - ‘VIW_FNT.csv’ saved [4371/4371]



In [ ]:
import pandas as pd
df = pd.read_csv("VIW_FNT.csv")

In [ ]:
df.shape

In [1]:
df.sample(5)

NameError: ignored

In [2]:
df.describe()

,ISO_YEAR,ISO_WEEK,MMWR_YEAR,MMWR_WEEK,SPEC_PROCESSED_NB,SPEC_RECEIVED_NB,AH1N12009,AH1,AH3,AH5,...,ADENO,BOCA,HUMAN_CORONA,METAPNEUMO,PARAINFLUENZA,RHINO,RSV,OTHERRESPVIRUS,ISOYW,MMWRYW
count,56419.000000,56419.000000,56419.000000,56419.000000,53602.000000,19491.000000,44109.000000,38870.000000,50116.000000,16984.000000,...,7242.000000,5336.000000,5136.000000,6586.000000,7293.000000,6173.000000,26262.000000,5945.000000,56418.000000,56418.000000
mean,2015.041032,26.160531,2015.039526,26.161541,460.983489,840.351342,15.053572,0.878107,11.329595,0.007183,...,6.786661,0.685907,4.174455,6.815518,9.370355,16.433177,15.770200,10.256182,201530.251427,201530.101776
std,6.402424,15.611284,6.403866,15.611054,3342.665627,5298.822857,166.107746,13.962122,106.447050,0.164063,...,21.707479,3.537126,24.313578,31.452069,34.939256,77.080739,94.315548,56.452488,640.199705,640.316665
min,1995.000000,1.000000,1995.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,199501.000000,199501.000000
25%,2011.000000,12.000000,2011.000000,12.000000,3.000000,11.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,201146.000000,201146.000000
50%,2016.000000,26.000000,2016.000000,26.000000,26.000000,38.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,201644.000000,201644.000000
75%,2020.000000,41.000000,2020.000000,41.000000,96.000000,116.000000,2.000000,0.000000,2.000000,0.000000,...,3.000000,0.000000,1.000000,2.000000,3.000000,4.000000,2.000000,0.000000,202039.000000,202039.000000
max,2023.000000,53.000000,2023.000000,53.000000,157960.000000,157960.000000,10575.000000,847.000000,12173.000000,14.000000,...,376.000000,102.000000,530.000000,723.000000,463.000000,1416.000000,3523.000000,704.000000,202350.000000,202350.000000


In [5]:
df.isnull().sum()

WHOREGION                      0
FLUSEASON                      0
HEMISPHERE                     0
ITZ                            0
COUNTRY_CODE                   0
COUNTRY_AREA_TERRITORY         0
ISO_WEEKSTARTDATE              0
ISO_YEAR                       0
ISO_WEEK                       0
MMWR_WEEKSTARTDATE             0
MMWR_YEAR                      0
MMWR_WEEK                      0
ORIGIN_SOURCE                  0
SPEC_PROCESSED_NB           2817
SPEC_RECEIVED_NB           36928
AH1N12009                  12310
AH1                        17549
AH3                         6303
AH5                        39435
AH7N9                      50057
ANOTSUBTYPED                7754
ANOTSUBTYPABLE             51666
AOTHER_SUBTYPE             40602
AOTHER_SUBTYPE_DETAILS     53851
INF_A                       4611
BVIC_2DEL                  53778
BVIC_3DEL                  54210
BVIC_NODEL                 14511
BVIC_DELUNK                53589
BYAM                       14940
BNOTDETERM

In [8]:
sumAH = df['AH1'].isnull().sum() + df['AH3'].isnull().sum() + df['AH5'].isnull().sum()
sumAH

63287

In [13]:
dfAH = df[['AH1','AH3','AH5']]
dfAH.corr()

,AH1,AH3,AH5
AH1,1.000000,0.09882,0.002948
AH3,0.098820,1.00000,-0.002190
AH5,0.002948,-0.00219,1.000000
